In [4]:
%load_ext autoreload

In [5]:
%autoreload 2

In [6]:
import sys
sys.path.append('../../../..')

In [7]:
import os
import time
import random
import argparse
import yaml
import cv2
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torchtext.data import metrics 
from nltk.metrics.distance import edit_distance
from data import build_vocab
from modules.converter import builder
from utils.predict_utils import resize
from utils.data_utils import post_process
from infer import prepare_model
from PIL import Image
import logging

In [8]:
config_path = '/media/huynhtruc0309/DATA/Math_Expression/my_source/Math_Recognition/config/test/cviu_experiments/ViGNN/experiment_0602.yaml'

In [9]:
config = yaml.load(open(config_path), Loader=yaml.SafeLoader)

In [10]:
print(config)

{'problem': 'math_recognition', 'imgH': None, 'imgW': None, 'max_width': 800, 'max_height': 800, 'min_width': 32, 'min_height': 32, 'batch_max_length': 500, 'rgb': False, 'pad': False, 'mean': 0.5, 'std': 0.5, 'beam_size': 1, 'vocab': '../data_20211227/vocab.txt', 'character': [], 'num_gpu': 0, 'batch_size': 0, 'workers': 0, 'postprocess': False, 'Transformation': 'None', 'FeatureExtraction': {'name': 'None'}, 'SequenceModeling': {'name': 'ViG', 'params': {'backbone': {'name': 'resnet', 'input_channel': 1, 'output_channel': 512, 'gcb': False}, 'input_channel': 1}}, 'Prediction': {'name': 'Attnv2', 'params': {'seqmodel': 'VIG', 'input_size': 256, 'hidden_size': 256, 'kernel_size': 2, 'kernel_dim': 128, 'embed_target': True, 'enc_init': True, 'attn_type': 'coverage', 'method': 'concat', 'teacher_forcing': 1.0, 'droprate': 0.25}}, 'export_csv': False, 'sanity_check': False, 'manualSeed': 1111, 'saved_model': '/media/huynhtruc0309/DATA/Math_Expression/my_source/Math_Recognition/saved_model

In [11]:
config['vocab'] = '/media/huynhtruc0309/DATA/Math_Expression/my_source/data_20211227/vocab.txt'
config['device'] = 'cpu'

In [12]:
tokenizer = build_vocab.build_tokenizer(config)
converter = builder.create_converter(config)
config['num_class'] = len(converter.character)
config['viz_attn'] = False

In [13]:
print(config['num_class'])

499


In [14]:
val_dir = '/media/huynhtruc0309/DATA/Math_Expression/my_source/data_20211115/updated_22011227/im2latex_validate_filter_updated_20220216.csv'

In [15]:
df = pd.read_csv(val_dir, sep=',', keep_default_na=False, names=['id', 'label'])

In [16]:
print(df.head(2))

               id                                              label
0  5abbb9b19f.png  \int _ { - \epsilon } ^ { \infty } d l \: \mat...
1  329a44c373.png  [ { \bar { K } } _ { a } ^ { - } ( p ) , { \ba...


In [17]:
data_dir = '/media/huynhtruc0309/DATA/Math_Expression/my_source/data_20211115/formula_images_processed'

In [18]:
print(len(df))

8370


In [19]:
test_sample = r'\int _ { - \epsilon } ^ { \infty } d l \: \mathrm { e } ^ { - l \zeta } \int _ { - \epsilon } ^ { \infty } d l ^ { \prime } \mathrm { e } ^ { - l ^ { \prime } \zeta } l l ^ { \prime } { \frac { l ^ { \prime } - l } { l + l ^ { \prime } } } \{ 3 \, \delta ^ { \prime \prime } ( l ) - { \frac { 3 } { 4 } } t \, \delta ( l ) \} = 0 .'

In [20]:
config['eval_data'] = data_dir
config['use_resizer'] = False

In [21]:
class TestDatasetSingle(Dataset):
    def __init__(self, df, opt, start_idx=0, has_label=False):
        test_df = df.copy()
        test_df = test_df.iloc[start_idx:]
        test_df.reset_index(drop=True, inplace=True)
        self.df = test_df
        self.opt = opt
        self.has_label=has_label
        self.resizer = None
        self.preprocess_time = 0
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if idx == len(self):
            return None, None, None 
        
        if self.has_label:
            img_name, label = self.df.loc[idx, 'id'], self.df.loc[idx, 'label']
            if len(label):
                if self.opt.get('token_level', 'word') == 'word':
                    label = [str(label).strip().split()]
                else: label = [str(label)]
        else:
            img_name = self.df.loc[idx, 'id']
        
        img_path=os.path.join(self.opt['eval_data'], str(img_name))
        start_time = time.time()
        new_img = resize(self.resizer, img_path, self.opt)
        end_time = time.time()
        pre_time = end_time - start_time
        self.preprocess_time += pre_time

        if self.has_label:
            return new_img, label, [img_name]
        else:
            return new_img, [img_name]

In [22]:
print(df.head(3))

               id                                              label
0  5abbb9b19f.png  \int _ { - \epsilon } ^ { \infty } d l \: \mat...
1  329a44c373.png  [ { \bar { K } } _ { a } ^ { - } ( p ) , { \ba...
2  73b51f198b.png  E ( v ) = \frac { d } { d t } E ( q ) \; \; \;...


In [23]:
dataset = TestDatasetSingle(df, config, start_idx=0, has_label=True)

In [24]:
model = prepare_model(config)

No Transformation module specified
------------ Options -------------
problem: math_recognition
imgH: None
imgW: None
max_width: 800
max_height: 800
min_width: 32
min_height: 32
batch_max_length: 500
rgb: False
pad: False
mean: 0.5
std: 0.5
beam_size: 1
vocab: /media/huynhtruc0309/DATA/Math_Expression/my_source/data_20211227/vocab.txt
character: ['}', '{', '_', '^', '2', '(', ')', '=', '1', '-', ',', '\\frac', '+', 'i', '0', 'x', 'n', '.', '\\,', 'd', 'a', '\\mu', 'e', 'k', 'm', 'r', 'c', 'p', '\\partial', '\\alpha', 't', 'A', '~', '\\;', '3', 'j', 's', 'l', '\\left(', '\\right)', 'g', '4', '\\', '\\nu', '\\prime', '\\pi', 'z', 'b', '\\phi', '|', '\\mathrm', '\\cal', '\\delta', 'f', 'N', 'q', '\\lambda', 'T', 'S', '\\beta', ']', 'R', '[', '\\bar', '\\int', 'D', 'M', 'L', '\\operatorname', 'B', 'F', '\\sigma', 'y', '&', '\\\\', '\\theta', '\\gamma', '\\psi', 'h', '/', '\\hat', '\\sqrt', 'H', '\\sum', 'u', '\\tilde', '\\rho', 'o', '\\tau', 'C', 'G', 'P', 'V', 'I', 'E', '\\omega', 'X', '\

In [27]:
from tools.interpretation.vit_visualize import show_mask_on_image, get_saliency_map

In [28]:
class Activation_Saver:
    def __init__(self, img_path, output_dir):
        self.img_path = img_path
        self.output_dir = output_dir
        self.down_sample = 1

    def __call__(self, save_name, save_obj):
        image = Image.open(self.img_path).convert('RGB')
        w, h = image.size
        new_h = int(h / self.down_sample)
        new_w = int(w / self.down_sample)
        
        if self.down_sample > 1:
            image = image.resize((new_w, new_h), Image.LANCZOS)
        else:
            image = image.resize((new_w, new_h), Image.BICUBIC)

        np_img = np.asarray(image).astype('uint8')
        
        resize_mask = F.interpolate(save_obj, (np_img.shape[0], np_img.shape[1]), mode='bicubic', align_corners=True)
        resize_mask = resize_mask.squeeze(0).squeeze(0)
        resize_mask =  np.asarray(resize_mask).astype(float)
        resize_mask = get_saliency_map(np_img, resize_mask)

        save_path = os.path.join(self.output_dir, os.path.basename(self.img_path).split('.')[0])
        if not os.path.exists(save_path): os.mkdir(save_path)
        cv2.imwrite(os.path.join(save_path, save_name), resize_mask)
        if not os.path.exists(os.path.join(save_path, os.path.basename(self.img_path))):
            image.save(os.path.join(save_path, os.path.basename(self.img_path)), format='PNG')

In [29]:
output_dir = '/media/huynhtruc0309/DATA/Math_Expression/my_source/Math_Recognition/analysis/cviu_analysis/model_analysis/vignn/experiment_2302'

In [30]:
def get_act_info(model, dataset, output_dir, num_process=1):
    logging.basicConfig(filename=os.path.join(output_dir, 'forward_logger.txt'),
                    filemode='w',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    force=True
    )

    forward_logger = logging.getLogger(name='Logger')

    model.eval()
    for idx, (image_tensors, _, img_name) in enumerate(dataset):
        if idx == num_process:
            break

        if image_tensors is None:
            break

        forward_logger.info(f'----------{img_name[0]}-----------')
        with torch.no_grad():
            image_tensors = image_tensors.to(config['device'])
            if not os.path.exists(os.path.join(output_dir, 'viz_output')): os.mkdir(os.path.join(output_dir, 'viz_output'))
            attnmap_viz = {
                'obj': Activation_Saver(
                    os.path.join(data_dir, img_name[0]), 
                    os.path.join(output_dir, 'viz_output')
                )
            }
            context_values, output_shapes = model.forward_encoder(image_tensors, debug=True, logger=forward_logger, attnmap_viz=attnmap_viz)

In [31]:
num_process=5
get_act_info(model, dataset, output_dir, num_process=num_process)